**라이브러리 import** 

In [2]:
from time import time
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook

**데이터 경로 설정 및 pandas로 불러오기**

In [ ]:
path = './_data\dacon_travle/' 
train_set = pd.read_csv(path + 'train.csv',index_col=0)
test_set = pd.read_csv(path + 'test.csv', index_col=0)

**결측치 처리**


Typeofcontact -> self Enquiry 로 대체 

In [ ]:
train_set['TypeofContact'].fillna('Self Enquiry', inplace=True)
test_set['TypeofContact'].fillna('Self Enquiry', inplace=True)

Age 컬럼은 직급에 따라 평균값으로 채워주기

In [ ]:
train_set['Age'].fillna(train_set.groupby('Designation')['Age'].transform('mean'), inplace=True)
test_set['Age'].fillna(test_set.groupby('Designation')['Age'].transform('mean'), inplace=True)
train_set['Age']=np.round(train_set['Age'],0).astype(int)
test_set['Age']=np.round(test_set['Age'],0).astype(int)

MonthlyIncome 의 결측치는 직급 컬럼 Designation 과 비교 ,차이를 보여 결측치를 대체

In [ ]:
train_set['MonthlyIncome'].fillna(train_set.groupby('Designation')['MonthlyIncome'].transform('mean'), inplace=True)
test_set['MonthlyIncome'].fillna(test_set.groupby('Designation')['MonthlyIncome'].transform('mean'), inplace=True)

NumberOfChildrenVisiting 컬럼은 결혼 여부에 따라 MaritalStatus 컬럼의 평균값으로 대체 NumberOfFollowups 컬럼은 평균값에 편차가 큰 컬럼의 평균값으로 대체

In [ ]:
train_set['NumberOfChildrenVisiting'].fillna(train_set.groupby('MaritalStatus')['NumberOfChildrenVisiting'].transform('mean'), inplace=True)
test_set['NumberOfChildrenVisiting'].fillna(test_set.groupby('MaritalStatus')['NumberOfChildrenVisiting'].transform('mean'), inplace=True)
train_set['NumberOfFollowups'].fillna(train_set.groupby('NumberOfChildrenVisiting')['NumberOfFollowups'].transform('mean'), inplace=True)
test_set['NumberOfFollowups'].fillna(test_set.groupby('NumberOfChildrenVisiting')['NumberOfFollowups'].transform('mean'), inplace=True)

DurationOfPitch 컬럼 '0'으로 대체

In [ ]:
train_set['DurationOfPitch']=train_set['DurationOfPitch'].fillna(0)
test_set['DurationOfPitch']=test_set['DurationOfPitch'].fillna(0)


PreferredPropertyStar 컬럼은 Occupation컬럼 평균값으로 대체 

In [ ]:
train_set['PreferredPropertyStar'].fillna(train_set.groupby('Occupation')['PreferredPropertyStar'].transform('mean'), inplace=True)
test_set['PreferredPropertyStar'].fillna(test_set.groupby('Occupation')['PreferredPropertyStar'].transform('mean'), inplace=True)

연속형 수치데이터인 Age 컬럼 범주화

In [ ]:
combine = [train_set,test_set]
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 20, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 20) & (dataset['Age'] <= 29), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 29) & (dataset['Age'] <= 39), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 39) & (dataset['Age'] <= 49), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 49) & (dataset['Age'] <= 59), 'Age'] = 4
    dataset.loc[ dataset['Age'] > 59, 'Age'] = 5


NumberOfTrips의 결측치는 DurationOfPitch의 평균으로 대체

In [ ]:
train_set['NumberOfTrips'].fillna(train_set.groupby('DurationOfPitch')['NumberOfTrips'].transform('mean'), inplace=True)
test_set['NumberOfTrips'].fillna(test_set.groupby('DurationOfPitch')['NumberOfTrips'].transform('mean'), inplace=True)

Occupation의 Free Lancer는 이상치로서 최빈값인 Salaried로 대체

In [ ]:
train_set.loc[ train_set['Occupation'] =='Free Lancer' , 'Occupation'] = 'Salaried'
test_set.loc[ test_set['Occupation'] =='Free Lancer' , 'Occupation'] = 'Salaried'

Fe male 컬럼을 Female로 결합 

In [ ]:
train_set.loc[ train_set['Gender'] =='Fe Male' , 'Gender'] = 'Female'
test_set.loc[ test_set['Gender'] =='Fe Male' , 'Gender'] = 'Female'

컬럼 정리, 라벨링

In [ ]:
cols = ['TypeofContact','Occupation','Gender','ProductPitched','MaritalStatus','Designation']


for col in tqdm_notebook(cols):
    le = LabelEncoder()
    train_set[col]=le.fit_transform(train_set[col])
    test_set[col]=le.fit_transform(test_set[col])


상관관계 낮은 컬럼들 drop 및 (train,test) 분리 , x,y 값 정의

In [ ]:
x = train_set.drop(['ProdTaken',
                          'NumberOfChildrenVisiting',
                          'NumberOfPersonVisiting',
                          'OwnCar', 
                          'MonthlyIncome', 
                          'NumberOfFollowups',

                          ], axis=1)

test_set = test_set.drop(['NumberOfChildrenVisiting',
                          'NumberOfPersonVisiting',
                          'OwnCar', 
                          'MonthlyIncome', 
                          'NumberOfFollowups',
                        #
                          ], axis=1)
y = train_set['ProdTaken']

x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.91,shuffle=True,random_state=1234,stratify=y)

교차검증 - StratifiedKFold
모델 - CatBoostClassifier
파라미터 튜닝 - RandomizedSearchCV 을 통해 성능이 가장 좋은 파라미터 설정

In [ ]:
n_splits = 6

kfold = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=123)

cat_paramets = {"learning_rate" : [0.01],
                'depth' : [8],
                'od_pval' : [0.12673190617341812],
                'fold_permutation_block': [142],
                'l2_leaf_reg' :[0.33021257848638497]}
cat = CatBoostClassifier(random_state=72,verbose=False,n_estimators=1304)
model = RandomizedSearchCV(cat,cat_paramets,cv=kfold,n_jobs=-1,)

import time 
start_time = time.time()
model.fit(x_train,y_train)   
end_time = time.time()-start_time 
y_predict = model.predict(x_test)
results = accuracy_score(y_test,y_predict)

model.fit(x,y)
y_submmit = model.predict(test_set)
y_submmit = np.round(y_submmit,0)
submission = pd.read_csv(path + 'sample_submission.csv',
                      )
submission['ProdTaken'] = y_submmit

submission.to_csv('test100.csv',index=False)

import pickle
path = '.\_save\_xg/'
pickle.dump(model, open(path + 'm39_pickle1_save', 'wb')) 
